In [7]:
import os
import glob
import re
import sys
import cv2

sys.path.append('../mediapipe')
from MediapipeKPDetector import MediapipeKPDetector


base_path = os.getcwd()
data_path = os.path.join(base_path, 'data')

image_extensions = ['jpg','jpeg', 'png']

images = [file_name for file_name in glob.glob(os.path.join(data_path, '*')) if re.match(r'\S+\.(jpg|jpeg|png)', file_name)]

for image_path in images:
    ground_truth_file = os.path.join(data_path, os.path.splitext(os.path.basename(image_path))[0] + '_ground_truth.txt')
    assert os.path.exists(ground_truth_file)
    
    gt_points = []
    
    with open(ground_truth_file, 'r') as gt:
        read = False
        for line in gt:
            if line.startswith('# 68'):
                read = True
            elif read and line.startswith('# }'):
                break
            elif read:
                # Split the string, remove the newline and convert the values to int
                gt_points.append(list(map(int, line.rstrip('\n').split(','))))
            
    assert len(gt_points) == 68
    
    # Detect using mediapipe
    detector = MediapipeKPDetector(maxFaces=1)
    
    img = cv2.imread(image_path)
    img, faces = detector.findFaceMesh(img, draw_points=False, draw_indices=False, filtered=True)
    
    assert len(faces[0]) == 51